<a href="https://colab.research.google.com/github/pocari-pixel/Modelling-and-Visualisation-Kelvin-Iria/blob/main/Modelling_and_Visualisation_Kelvin_Iria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import files
uploaded = files.upload()

Saving CreditCardDataFinal.csv to CreditCardDataFinal.csv


In [6]:
#Packages related to general operating system & warnings
import os 
import warnings
warnings.filterwarnings('ignore')
#Packages related to data importing, manipulation, exploratory data #analysis, data understanding
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import datetime as datetime
from termcolor import colored as cl # text customization
#Packages related to data visualizaiton
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
#Setting plot sizes and type of plot
plt.rc("font", size=14)
plt.rcParams['axes.grid'] = True
plt.figure(figsize=(6,3))
plt.gray()
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.impute import MissingIndicator, SimpleImputer
from sklearn.preprocessing import  PolynomialFeatures, KBinsDiscretizer, FunctionTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer, OrdinalEncoder
import statsmodels.formula.api as smf
import statsmodels.tsa as tsa
from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.ensemble import BaggingClassifier, BaggingRegressor,RandomForestClassifier,RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier,GradientBoostingRegressor, AdaBoostClassifier, AdaBoostRegressor 
from sklearn.svm import LinearSVC, LinearSVR, SVC, SVR

<Figure size 432x216 with 0 Axes>

In [7]:
data = pd.read_csv('CreditCardDataFinal.csv')
data.head()

,Transaction ID,Date,Day of Week,Time,Type of Card,Entry Mode,Amount,Type of Transaction,Merchant Group,Country of Transaction,Shipping Address,Country of Residence,Gender,Age,Bank,Fraud
0,#3577 209,14-Oct-20,Wednesday,19,Visa,Tap,£5,POS,Entertainment,United Kingdom,United Kingdom,United Kingdom,M,25.2,RBS,0
1,#3039 221,14-Oct-20,Wednesday,17,MasterCard,PIN,£288,POS,Services,USA,USA,USA,F,49.6,Lloyds,0
2,#2694 780,14-Oct-20,Wednesday,14,Visa,Tap,£5,POS,Restaurant,India,India,India,F,42.2,Barclays,0
3,#2640 960,13-Oct-20,Tuesday,14,Visa,Tap,£28,POS,Entertainment,United Kingdom,India,United Kingdom,F,51.0,Barclays,0
4,#2771 031,13-Oct-20,Tuesday,23,Visa,CVC,£91,Online,Electronics,USA,USA,United Kingdom,M,38.0,Halifax,1


In [8]:
Total_transactions = len(data)
normal = len(data[data.Fraud == 0])
fraudulent = len(data[data.Fraud == 1])
fraud_percentage = round(fraudulent/normal*100, 2)
print(cl('Total number of Trnsactions are {}'.format(Total_transactions), attrs = ['bold']))
print(cl('Number of Normal Transactions are {}'.format(normal), attrs = ['bold']))
print(cl('Number of fraudulent Transactions are {}'.format(fraudulent), attrs = ['bold']))
print(cl('Percentage of fraud Transactions is {}'.format(fraud_percentage), attrs = ['bold']))


Total number of Trnsactions are 100000
Number of Normal Transactions are 92805
Number of fraudulent Transactions are 7195
Percentage of fraud Transactions is 7.75


In [9]:
data['Transaction ID'] = data['Transaction ID'].str.replace('#','')
data['Transaction ID'] = data['Transaction ID'].str.replace(' ','')
data['Amount'] = data['Amount'].str.replace('£','')
data.head()

,Transaction ID,Date,Day of Week,Time,Type of Card,Entry Mode,Amount,Type of Transaction,Merchant Group,Country of Transaction,Shipping Address,Country of Residence,Gender,Age,Bank,Fraud
0,3577209,14-Oct-20,Wednesday,19,Visa,Tap,5,POS,Entertainment,United Kingdom,United Kingdom,United Kingdom,M,25.2,RBS,0
1,3039221,14-Oct-20,Wednesday,17,MasterCard,PIN,288,POS,Services,USA,USA,USA,F,49.6,Lloyds,0
2,2694780,14-Oct-20,Wednesday,14,Visa,Tap,5,POS,Restaurant,India,India,India,F,42.2,Barclays,0
3,2640960,13-Oct-20,Tuesday,14,Visa,Tap,28,POS,Entertainment,United Kingdom,India,United Kingdom,F,51.0,Barclays,0
4,2771031,13-Oct-20,Tuesday,23,Visa,CVC,91,Online,Electronics,USA,USA,United Kingdom,M,38.0,Halifax,1


In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Transaction ID          100000 non-null  object 
 1   Date                    100000 non-null  object 
 2   Day of Week             100000 non-null  object 
 3   Time                    100000 non-null  int64  
 4   Type of Card            100000 non-null  object 
 5   Entry Mode              100000 non-null  object 
 6   Amount                  99994 non-null   object 
 7   Type of Transaction     100000 non-null  object 
 8   Merchant Group          99990 non-null   object 
 9   Country of Transaction  100000 non-null  object 
 10  Shipping Address        99995 non-null   object 
 11  Country of Residence    100000 non-null  object 
 12  Gender                  99996 non-null   object 
 13  Age                     100000 non-null  float64
 14  Bank                 

In [11]:
data=data.convert_dtypes()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   Transaction ID          100000 non-null  string 
 1   Date                    100000 non-null  string 
 2   Day of Week             100000 non-null  string 
 3   Time                    100000 non-null  Int64  
 4   Type of Card            100000 non-null  string 
 5   Entry Mode              100000 non-null  string 
 6   Amount                  99994 non-null   string 
 7   Type of Transaction     100000 non-null  string 
 8   Merchant Group          99990 non-null   string 
 9   Country of Transaction  100000 non-null  string 
 10  Shipping Address        99995 non-null   string 
 11  Country of Residence    100000 non-null  string 
 12  Gender                  99996 non-null   string 
 13  Age                     100000 non-null  Float64
 14  Bank                 

In [12]:
data['Date']=pd.to_datetime(data['Date'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 16 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   Transaction ID          100000 non-null  string        
 1   Date                    100000 non-null  datetime64[ns]
 2   Day of Week             100000 non-null  string        
 3   Time                    100000 non-null  Int64         
 4   Type of Card            100000 non-null  string        
 5   Entry Mode              100000 non-null  string        
 6   Amount                  99994 non-null   string        
 7   Type of Transaction     100000 non-null  string        
 8   Merchant Group          99990 non-null   string        
 9   Country of Transaction  100000 non-null  string        
 10  Shipping Address        99995 non-null   string        
 11  Country of Residence    100000 non-null  string        
 12  Gender                  99996 n

In [15]:
data['Age'].max()

86.1

In [ ]:
data = data.astype({})

In [27]:
data.head()

,Transaction ID,Date,Day of Week,Time,Type of Card,Entry Mode,Amount,Type of Transaction,Merchant Group,Country of Transaction,Shipping Address,Country of Residence,Gender,Age,Bank,Fraud
0,3577209,2020-10-14,Wednesday,19,Visa,Tap,5,POS,Entertainment,United Kingdom,United Kingdom,United Kingdom,M,25.2,RBS,0
1,3039221,2020-10-14,Wednesday,17,MasterCard,PIN,288,POS,Services,USA,USA,USA,F,49.6,Lloyds,0
2,2694780,2020-10-14,Wednesday,14,Visa,Tap,5,POS,Restaurant,India,India,India,F,42.2,Barclays,0
3,2640960,2020-10-13,Tuesday,14,Visa,Tap,28,POS,Entertainment,United Kingdom,India,United Kingdom,F,51.0,Barclays,0
4,2771031,2020-10-13,Tuesday,23,Visa,CVC,91,Online,Electronics,USA,USA,United Kingdom,M,38.0,Halifax,1


In [18]:
min(data.Amount), max(data.Amount)

TypeError: ignored

In [167]:
df['Transaction ID'].is_unique

False

In [168]:
df.dtypes

Transaction ID             object
Date                       object
Day of Week                object
Time                        int64
Type of Card               object
Entry Mode                 object
Amount                     object
Type of Transaction        object
Merchant Group             object
Country of Transaction     object
Shipping Address           object
Country of Residence       object
Gender                     object
Age                       float64
Bank                       object
Fraud                       int64
dtype: object

In [170]:
df = df.astype({'Transaction ID': int})

In [171]:
df.dtypes

Transaction ID              int64
Date                       object
Day of Week                object
Time                        int64
Type of Card               object
Entry Mode                 object
Amount                     object
Type of Transaction        object
Merchant Group             object
Country of Transaction     object
Shipping Address           object
Country of Residence       object
Gender                     object
Age                       float64
Bank                       object
Fraud                       int64
dtype: object

In [172]:
df = df.set_index('Transaction ID', inplace=True)

In [179]:
df.isnull().sum()

AttributeError: ignored

In [178]:
df.get_dtype_counts()

AttributeError: ignored